In [2]:
%cd /home/viresh16118/repos/jodie/

/home/viresh16118/repos/jodie


In [5]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [129]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from collections import defaultdict
import dyna_loader as dnldr

In [8]:
[user2id, user_sequence_id, 
 user_timedifference_sequence, 
 user_previous_itemid_sequence, 
 op_sequence_id, 
 op_timedifference_sequence, 
 item2id, item_sequence_id, 
 item_timedifference_sequence, 
 timestamp_sequence, 
 feature_sequence, 
 y_true_labels] = dnldr.load_network()

Loading network from /home/viresh16118/ICDM_Computed/stackoverflow/feature_extracted.pkl and /home/viresh16118/ICDM_Computed/stackoverflow/id_to_body.pkl
Formating item sequence
Formating user sequence
Formatting original poster sequence
Scaling Timestamps
*** Network loading completed ***




Check what actually is inside jodie's contents

In [9]:
sum(y_true_labels), len(y_true_labels)

(1397086.0, 1397086)

In [11]:
num_interactions = len(user_sequence_id)
num_users = len(user2id) 
num_items = len(item2id) + 1 # one extra item for "none-of-these"
num_features = len(feature_sequence[0])
print("*** Network statistics:\n  %d users\n  %d items\n  %d interactions\n  %d feature size ***\n\n" % (num_users, num_items, num_interactions, num_features))

*** Network statistics:
  90182 users
  19911 items
  1397086 interactions
  73 feature size ***




In [12]:
train_proportion = 0.8
train_end_idx = validation_start_idx = int(num_interactions * train_proportion) 
test_start_idx = int(num_interactions * (train_proportion+0.1))
test_end_idx = int(num_interactions * (train_proportion+0.2))

timespan = timestamp_sequence[-1] - timestamp_sequence[0]
tbatch_timespan = timespan / 500 
print(tbatch_timespan)

5355.894


In [26]:
embedding_dim = 128
initial_userembeddings = F.normalize(torch.randn(embedding_dim), dim=0)
initial_itemembeddings = F.normalize(torch.randn(embedding_dim), dim=0)

user_embeddings = initial_userembeddings.repeat(num_users, 1)
item_embeddings = initial_userembeddings.repeat(num_items, 1)

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

lib = dotdict({
    # list of users of each tbatch up to now
    'current_tbatches_interactionids': defaultdict(list),
    'current_tbatches_user': defaultdict(list),
    'current_tbatches_item': defaultdict(list),
    'current_tbatches_timestamp': defaultdict(list),
    'current_tbatches_feature': defaultdict(list),
    'current_tbatches_label': defaultdict(list),
    'current_tbatches_previous_item': defaultdict(list),
    'current_tbatches_user_timediffs': defaultdict(list),
    'current_tbatches_item_timediffs': defaultdict(list),
    'current_tbatches_user_timediffs_next': defaultdict(list),
    
    # the latest tbatch a user is in
    'tbatchid_user': defaultdict(lambda: -1),

    # the latest tbatch a item is in
    'tbatchid_item': defaultdict(lambda: -1),
})
print(lib.tbatchid_user)

defaultdict(<function <lambda> at 0x7f0c3db61c80>, {})


In [29]:
tbatch_start_time = None
tbatch_to_insert = -1
tbatch_full = False

for j in range(train_end_idx):
    # READ INTERACTION J
    userid = user_sequence_id[j]
    itemid = item_sequence_id[j]
    feature = feature_sequence[j]
    user_timediff = user_timedifference_sequence[j]
    item_timediff = item_timedifference_sequence[j]

    # CREATE T-BATCHES: ADD INTERACTION J TO THE CORRECT T-BATCH
    tbatch_to_insert = max(lib.tbatchid_user[userid], lib.tbatchid_item[itemid]) + 1 
    lib.tbatchid_user[userid] = tbatch_to_insert 
    lib.tbatchid_item[itemid] = tbatch_to_insert

    lib.current_tbatches_user[tbatch_to_insert].append(userid)
    lib.current_tbatches_item[tbatch_to_insert].append(itemid)
    lib.current_tbatches_feature[tbatch_to_insert].append(feature)
    lib.current_tbatches_interactionids[tbatch_to_insert].append(j)
    lib.current_tbatches_user_timediffs[tbatch_to_insert].append(user_timediff)
    lib.current_tbatches_item_timediffs[tbatch_to_insert].append(item_timediff)
    lib.current_tbatches_previous_item[tbatch_to_insert].append(user_previous_itemid_sequence[j])

    timestamp = timestamp_sequence[j]
    if tbatch_start_time is None:
        tbatch_start_time = timestamp

    # AFTER ALL INTERACTIONS IN THE TIMESPAN ARE CONVERTED TO T-BATCHES, FORWARD PASS TO CREATE EMBEDDING TRAJECTORIES AND CALCULATE PREDICTION LOSS
    if timestamp - tbatch_start_time > tbatch_timespan:
        break
        tbatch_start_time = timestamp # RESET START TIME FOR THE NEXT TBATCHES

#         # ITERATE OVER ALL T-BATCHES
#         with trange(len(lib.current_tbatches_user)) as progress_bar3:
#             for i in progress_bar3:
#                 progress_bar3.set_description('Processed %d of %d T-batches ' % (i, len(lib.current_tbatches_user)))

#                 total_interaction_count += len(lib.current_tbatches_interactionids[i])

#                 # LOAD THE CURRENT TBATCH
#                 tbatch_userids = torch.LongTensor(lib.current_tbatches_user[i]).cuda() # Recall "lib.current_tbatches_user[i]" has unique elements
#                 tbatch_itemids = torch.LongTensor(lib.current_tbatches_item[i]).cuda() # Recall "lib.current_tbatches_item[i]" has unique elements
#                 tbatch_interactionids = torch.LongTensor(lib.current_tbatches_interactionids[i]).cuda() 
#                 feature_tensor = Variable(torch.Tensor(lib.current_tbatches_feature[i]).cuda()) # Recall "lib.current_tbatches_feature[i]" is list of list, so "feature_tensor" is a 2-d tensor
#                 user_timediffs_tensor = Variable(torch.Tensor(lib.current_tbatches_user_timediffs[i]).cuda()).unsqueeze(1)
#                 item_timediffs_tensor = Variable(torch.Tensor(lib.current_tbatches_item_timediffs[i]).cuda()).unsqueeze(1)
#                 tbatch_itemids_previous = torch.LongTensor(lib.current_tbatches_previous_item[i]).cuda()
#                 item_embedding_previous = item_embeddings[tbatch_itemids_previous,:]

#                 # PROJECT USER EMBEDDING TO CURRENT TIME
#                 user_embedding_input = user_embeddings[tbatch_userids,:]
#                 user_projected_embedding = model.forward(user_embedding_input, item_embedding_previous, timediffs=user_timediffs_tensor, features=feature_tensor, select='project')
#                 user_item_embedding = torch.cat([user_projected_embedding, item_embedding_previous, item_embedding_static[tbatch_itemids_previous,:], user_embedding_static[tbatch_userids,:]], dim=1)

#                 # PREDICT NEXT ITEM EMBEDDING                            
#                 predicted_item_embedding = model.predict_item_embedding(user_item_embedding)

#                 # CALCULATE PREDICTION LOSS
#                 item_embedding_input = item_embeddings[tbatch_itemids,:]
#                 loss += MSELoss(predicted_item_embedding, torch.cat([item_embedding_input, item_embedding_static[tbatch_itemids,:]], dim=1).detach())

#                 # UPDATE DYNAMIC EMBEDDINGS AFTER INTERACTION
#                 user_embedding_output = model.forward(user_embedding_input, item_embedding_input, timediffs=user_timediffs_tensor, features=feature_tensor, select='user_update')
#                 item_embedding_output = model.forward(user_embedding_input, item_embedding_input, timediffs=item_timediffs_tensor, features=feature_tensor, select='item_update')

#                 item_embeddings[tbatch_itemids,:] = item_embedding_output
#                 user_embeddings[tbatch_userids,:] = user_embedding_output  

#                 user_embeddings_timeseries[tbatch_interactionids,:] = user_embedding_output
#                 item_embeddings_timeseries[tbatch_interactionids,:] = item_embedding_output

#                 # CALCULATE LOSS TO MAINTAIN TEMPORAL SMOOTHNESS
#                 loss += MSELoss(item_embedding_output, item_embedding_input.detach())
#                 loss += MSELoss(user_embedding_output, user_embedding_input.detach())

#                 # CALCULATE STATE CHANGE LOSS
#                 if args.state_change:
#                     loss += calculate_state_prediction_loss(model, tbatch_interactionids, user_embeddings_timeseries, y_true, crossEntropyLoss) 

#         # BACKPROPAGATE ERROR AFTER END OF T-BATCH
#         total_loss += loss.item()
#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()

#         # RESET LOSS FOR NEXT T-BATCH
#         loss = 0
#         item_embeddings.detach_() # Detachment is needed to prevent double propagation of gradient
#         user_embeddings.detach_()
#         item_embeddings_timeseries.detach_() 
#         user_embeddings_timeseries.detach_()

#         # REINITIALIZE
#         reinitialize_tbatches()
#         tbatch_to_insert = -1


In [31]:
print(len(lib.current_tbatches_user), len(lib.current_tbatches_item), len(lib.current_tbatches_interactionids))

55 55 55


In [40]:
i = 0
tbatch_userids = torch.LongTensor(lib.current_tbatches_user[i])
tbatch_itemids = torch.LongTensor(lib.current_tbatches_item[i])
tbatch_interactionids = torch.LongTensor(lib.current_tbatches_interactionids[i]) 
feature_tensor = (torch.Tensor(lib.current_tbatches_feature[i]))
user_timediffs_tensor = (torch.Tensor(lib.current_tbatches_user_timediffs[i])).unsqueeze(1)
item_timediffs_tensor = (torch.Tensor(lib.current_tbatches_item_timediffs[i])).unsqueeze(1)
tbatch_itemids_previous = torch.LongTensor(lib.current_tbatches_previous_item[i])
item_embedding_previous = item_embeddings[tbatch_itemids_previous,:]
print(tbatch_userids.shape)

torch.Size([31])


In [53]:
user_embedding_input = user_embeddings[tbatch_userids,:]
item_embedding_input = item_embeddings[tbatch_itemids,:]
print(user_embedding_input.size())
print(item_embedding_input.size())
print(user_timediffs_tensor.size())
print(item_timediffs_tensor.size())
print(feature_tensor.size())

torch.Size([31, 128])
torch.Size([31, 128])
torch.Size([31, 1])
torch.Size([31, 1])
torch.Size([31, 73])


In [49]:
output = []
for ue, ie, ut, it in zip(torch.unbind(user_embedding_input, 0), torch.unbind(item_embedding_input, 0), torch.unbind(user_timediffs_tensor, 0), torch.unbind(item_timediffs_tensor, 0)):
    print(ue.size(), ie.size(), ut.size(), it.size())
    output.append(torch.randn(128))
output = torch.stack(output, dim=0)

torch.Size([128]) torch.Size([128]) torch.Size([1]) torch.Size([1])
torch.Size([128]) torch.Size([128]) torch.Size([1]) torch.Size([1])
torch.Size([128]) torch.Size([128]) torch.Size([1]) torch.Size([1])
torch.Size([128]) torch.Size([128]) torch.Size([1]) torch.Size([1])
torch.Size([128]) torch.Size([128]) torch.Size([1]) torch.Size([1])
torch.Size([128]) torch.Size([128]) torch.Size([1]) torch.Size([1])
torch.Size([128]) torch.Size([128]) torch.Size([1]) torch.Size([1])
torch.Size([128]) torch.Size([128]) torch.Size([1]) torch.Size([1])
torch.Size([128]) torch.Size([128]) torch.Size([1]) torch.Size([1])
torch.Size([128]) torch.Size([128]) torch.Size([1]) torch.Size([1])
torch.Size([128]) torch.Size([128]) torch.Size([1]) torch.Size([1])
torch.Size([128]) torch.Size([128]) torch.Size([1]) torch.Size([1])
torch.Size([128]) torch.Size([128]) torch.Size([1]) torch.Size([1])
torch.Size([128]) torch.Size([128]) torch.Size([1]) torch.Size([1])
torch.Size([128]) torch.Size([128]) torch.Size([

In [50]:
output.size()

torch.Size([31, 128])

In [51]:
user_embedding_input.size()

torch.Size([31, 128])

Test dynamic embedding model

In [186]:
class NormalLinear(nn.Linear):
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.normal_(0, stdv)
        if self.bias is not None:
            self.bias.data.normal_(0, stdv)

class CustomCell(nn.Module):
    def __init__(self, embedding_size, feature_size):
        super(CustomCell, self).__init__()
        self.embsz = embedding_size
        self.uembsz = embedding_size
        self.iembsz = embedding_size
        self.featsz = feature_size
        self.N = 3*embedding_size + feature_size + 1
        self.WdynU1 = torch.nn.Parameter(torch.randn(embedding_size, self.uembsz, self.N))
        self.WdynU2 = torch.nn.Parameter(torch.randn(embedding_size, self.uembsz, self.N))
        self.WdynI1 = torch.nn.Parameter(torch.randn(embedding_size, self.iembsz, self.N))
        self.WdynF  = torch.nn.Parameter(torch.randn(embedding_size, self.featsz, self.N))
        self.WdynT  = torch.nn.Parameter(torch.randn(embedding_size, 1, self.N))
        
        ### this parameter remains unused in ItemRNN. Wastes GPU Space.
        ### Probably find some better place for this?
        self.WdynP = torch.nn.Parameter(torch.randn(embedding_size, self.uembsz, self.N))
    
    def forward(self, U1, U2, I1, F, T):
        ntensor = torch.cat([U1, U2, I1, F, T]).unsqueeze(1)
        W1 = torch.mm(self.WdynU1.view(self.embsz * self.uembsz, -1), ntensor).view(self.embsz, self.uembsz, -1)
        W2 = torch.mm(self.WdynU2.view(self.embsz * self.uembsz, -1), ntensor).view(self.embsz, self.uembsz, -1)
        W3 = torch.mm(self.WdynI1.view(self.embsz * self.iembsz, -1), ntensor).view(self.embsz, self.iembsz, -1)
        W4 = torch.mm(self.WdynF.view(self.embsz * self.featsz, -1), ntensor).view(self.embsz, self.featsz, -1)
        W5 = torch.mm(self.WdynT.view(self.embsz * 1, -1), ntensor).view(self.embsz, 1, -1)

        ### All N_i's are of same size
        N_1 = torch.tanh(torch.mm(W1.squeeze(2), U1.unsqueeze(1)))
        N_2 = torch.tanh(torch.mm(W2.squeeze(2), U2.unsqueeze(1)))
        N_3 = torch.tanh(torch.mm(W3.squeeze(2), I1.unsqueeze(1)))
        N_4 = torch.tanh(torch.mm(W4.squeeze(2), F.unsqueeze(1)))
        N_5 = torch.tanh(torch.mm(W5.squeeze(2), T.unsqueeze(1)))
        
        ### Perform predictions over this interaction
        user_predicted, item_predicted = self.predict_(U1, U2, I1, F, T)
        
        # return the sum of all N_i's as new embedding
        return torch.sum(torch.stack([N_1, N_2, N_3, N_4, N_5], dim=1), dim=1), user_predicted, item_predicted
    
    def project_user(self, U1_, U2_, I1_, F_, delta):
        # Predict embedding of user U1 from it's embedding at U_ at T-
        ntensor = torch.cat([U1_, U2_, I1_, F_, delta]).unsqueeze(1)
        W5 = torch.mm(self.WdynT.view(self.embsz * 1, -1), ntensor).view(self.embsz, 1, -1)
        N_5 = torch.tanh(torch.mm(W5.squeeze(2), delta.unsqueeze(1)))
        predicted_u = (1. + N_5).squeeze(1) * U1_
        return predicted_u
    
    def predict_(self, U1_, U2_, I1_, F_, delta):
        user_hat = self.project_user(U1_, U2_, I1_, F_, delta)
        ntensor = torch.cat([U1_, U2_, I1_, F_, delta]).unsqueeze(1)
        W6 = torch.mm(self.WdynP.view(self.embsz * self.uembsz, -1), ntensor).view(self.embsz, self.uembsz, -1)
        item_hat = torch.tanh(torch.mm(W6.squeeze(2), user_hat.unsqueeze(1)))
        return user_hat, item_hat
    

class Wrapper(nn.Module):
    def __init__(self, emb_sz, feat_sz):
        super(Wrapper, self).__init__()
        self.cell = CustomCell(emb_sz, feat_sz)
    
    def forward(self, BU1, BU2, BI1, BF, BT):
        ### Input is batched. Unroll the TBatch and train for every timestep
        output = []
        user_predicted = []
        item_predicted = []
        for ue, oe, ie, fv, delta in zip(torch.unbind(BU1, 0), torch.unbind(BU2, 0), torch.unbind(BI1, 0), torch.unbind(BF, 0), torch.unbind(BT, 0)):
            new_embedding, u_predicted, i_predicted = self.cell(ue, oe, ie, fv, delta)
            output.append(new_embedding)
            user_predicted.append(u_predicted)
            item_predicted.append(i_predicted)
        output = torch.stack(output, dim=0).squeeze()
        user_predicted = torch.stack(user_predicted, dim=0).squeeze()
        item_predicted = torch.stack(item_predicted, dim=0).squeeze()
        return output, user_predicted, item_predicted

class DynaEmbedModel(nn.Module):
    def __init__(self, emb_sz, feat_sz):
        super(DynaEmbedModel, self).__init__()
        print("*** Initializing the DYNAEMBED model ***")
        self.userrnn = Wrapper(emb_sz, feat_sz)
        self.itemrnn = Wrapper(emb_sz, feat_sz)
        print("*** DYNAEMBED initialization complete ***\n\n")
    
    def forward(self, BU1, BU2, BI1, BF, BT, perform='predict'):
        # return new user, item and other user embedding
        if perform=='interacting_user':
            BU1_new, BU1_predicted, I1_predicted = self.userrnn(BU1, BU2, BI1, BF, BT)
            return BU1_new, BU1_predicted, I1_predicted
        elif perform == 'interacted_user':
            BU2_new, _, __ = self.userrnn(BU2, BU1, BI1, BF, BT)
            return BU2_new
        elif perform == 'item':
            BI1_new, _, __ = self.itemrnn(BU1, BU2, BI1, BF, BT)
            return BI1_new
        elif perform=='predict':
            _, BU1_predicted, I1_predicted = self.userrnn(BU1, BU2, BI1, BF, BT)
            return BU1_predicted, I1_predicted


In [187]:
sample_user_embedding_input = torch.randn(31, 128)
sample_item_embedding_input = torch.randn(31, 128)
sample_op_embedding_input   = torch.randn(31, 128)
sample_feature_input        = torch.randn(31, 73)
sample_time_diff            = torch.randn(31, 1)

# model = Wrapper(128, 73)
model = DynaEmbedModel(128, 73)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### perform one step for the sample tbatch user update
optimizer.zero_grad()

### Forward pass
output, _2, _3 = model(sample_user_embedding_input, sample_op_embedding_input, sample_item_embedding_input, sample_feature_input, sample_time_diff, 'interacting_user')

### Compute loss
loss = criterion(output, sample_user_embedding_input)

### Backpropagation (through time?)
loss.backward()

### Update parameters
optimizer.step()
print(loss.item())

*** Initializing the DYNAEMBED model ***
*** DYNAEMBED initialization complete ***


5.725464820861816


In [185]:
print(output.size())
print(_2.size())
print(_3.size())
print(_4.size())
print(_5.size())

torch.Size([31, 128])
torch.Size([31, 128])
torch.Size([31, 128])
torch.Size([31, 128])
torch.Size([31, 128])
